In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import zipfile
import shutil

FER_ZIP_PATH = 'fer-2013.zip.zip'
UNZIP_DESTINATION = '/content/extracted_fer_data'
IMG_SIZE = (48, 48)
NUM_CLASSES = 7
BATCH_SIZE = 64
EPOCHS = 30
print("Starting Data Extraction & Preprocessing")

if os.path.exists(UNZIP_DESTINATION):
    shutil.rmtree(UNZIP_DESTINATION)
os.makedirs(UNZIP_DESTINATION, exist_ok=True)
print(f"Destination created: {UNZIP_DESTINATION}")

try:
    print(f"Attempt to extract {FER_ZIP_PATH}...")
    with zipfile.ZipFile(FER_ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(UNZIP_DESTINATION)
    print("Extracting successful!!!")
except FileNotFoundError:
    print(f"File Not Found")
    exit()
extracted_contents = os.listdir(UNZIP_DESTINATION)
if len(extracted_contents) == 1 and os.path.isdir(os.path.join(UNZIP_DESTINATION, extracted_contents[0])):
    FER_BASE_DIR = os.path.join(UNZIP_DESTINATION, extracted_contents[0])
else:
    FER_BASE_DIR = UNZIP_DESTINATION

TRAIN_DIR = os.path.join(FER_BASE_DIR, 'train')
TEST_DIR = os.path.join(FER_BASE_DIR, 'test')
VAL_DIR = os.path.join(FER_BASE_DIR, 'validation')

if not os.path.exists(TRAIN_DIR):
    print(f"Training Directory Not Found")
    exit()
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode='grayscale', class_mode='categorical'
)
validation_dir = VAL_DIR if os.path.exists(VAL_DIR) else TEST_DIR

validation_generator = test_datagen.flow_from_directory(
    validation_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode='grayscale', class_mode='categorical'
)
print("Preprocessing Process Completed Successfully!!!")
print("Finally Milestone-1 Completed")

model = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])


callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Model Summary ---")
model.summary()
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)
MODEL_SAVE_PATH = 'emotion_detection_cnn_model.keras'
model.save(MODEL_SAVE_PATH)
print(f"\n Milestone 2 Output Saved: Model saved to {MODEL_SAVE_PATH}")
loaded_model = tf.keras.models.load_model('emotion_detection_cnn_model.keras')
print(" Milestone 2 Completed!!!")

Starting Data Extraction & Preprocessing
Destination created: /content/extracted_fer_data
Attempt to extract fer-2013.zip.zip...
Extracting successful!!!
Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Preprocessing Process Completed Successfully!!!
Finally Milestone-1 Completed

--- Model Summary ---


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 48, 48, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 24, 24, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 12, 12, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │         1,79

 Total params: 1,469,927 (5.61 MB)

 Trainable params: 1,469,031 (5.60 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 633s 1s/step - accuracy: 0.2343 - loss: 2.2602 - val_accuracy: 0.2581 - val_loss: 1.7913 - learning_rate: 0.0010
Epoch 2/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.2500 - loss: 1.7893 - val_accuracy: 0.2573 - val_loss: 1.7925 - learning_rate: 0.0010
Epoch 3/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - accuracy: 0.3038 - loss: 1.7231 - val_accuracy: 0.4174 - val_loss: 1.5059 - learning_rate: 0.0010
Epoch 4/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - accuracy: 0.3125 - loss: 1.6918 - val_accuracy: 0.4308 - val_loss: 1.4870 - learning_rate: 0.0010
Epoch 5/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 679s 1s/step - accuracy: 0.3531 - loss: 1.6120 - val_accuracy: 0.4009 - val_loss: 1.5401 - learning_rate: 0.0010
Epoch 6/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.4219 - loss: 1.5883 - val_accuracy: 0.4093 - val_loss: 1.5331 - learning_rate: 0.0010
Epoch 7/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 650s 1s/step - accuracy: 0.3895 - loss: